In [ ]:
from algorithms import kaczrank, cautiousrank, plot_average_w_ci, kr_itercount, noisykaczrank
import numpy as np
import matplotlib.pyplot as plt
import time

plt.rcParams.update({'font.size': 18})

from tqdm.auto import tqdm
from scipy.stats import rankdata
%matplotlib inline
from choix import *
import numba
from numba import jit

def get_ranking(vec):
    """Return the ranking associated with a vector of ratings."""
    return(rankdata(vec)-1).astype(int)

def hamdist(v,w):
    """Returns the Hamming distance between two vectors."""
    return np.sum(np.abs(v-w) > 1e-10)

def k_dist(v,w,k):
    """Returns the k-distance between two vectors."""
    return np.sum(np.abs(v-w) > k + 1e-10)

In [ ]:
def build_adj_mat(true_rank, make_comps = False, p=0):
    """
    Build incidence matrix for pairwise comparisons.
    
    Parameters:
    true_rank: array, vector of true rankings from which to build an incidence matrix.
    make_comps: bool, whether to also return list of pairwise comparisons.
    p: float, probability of incidence matrix containing flipped pairs.
    
    Returns:
    A: array, incidence matrix
    comps (if make_comps): list of pairwise comparisons
    """
    n = len(true_rank)

    A = np.zeros((int(n*(n-1)/2)+1, n))

    t=0
    comps = []
    for i in tqdm(range(n),leave=False):
        for j in range(i+1,n):
            
            flip = 1
            if np.random.rand() < p:
                flip = -1
            
            if flip*true_rank[i] >= flip*true_rank[j]:
                
                A[t,i] = -1
                A[t,j] = 1
                t+=1

                if make_comps:
                    comps.append((j,i))
            else:

                A[t,i] = 1
                A[t,j] = -1
                
                t+=1
                
                if make_comps:
                    comps.append((i,j))

    if make_comps:
        return A, comps
       
    return A

In [ ]:
## Experiment 1 : KaczRank applied to a full information system with no noise.

n = 50
n_iters = 10000
n_trials = 20
eps = 1e-5
true_vals = np.random.rand(n)
true_rank = get_ranking(true_vals)
A, comps = build_adj_mat(true_rank, make_comps = True)
x_init=np.random.rand(n)


In [ ]:
ham_results = []

k1dist_results = []
k5dist_results = []
k10dist_results = []

ls_results = []
skr_results = []
for i in tqdm(range(n_trials)):
    iters, ranks = kaczrank(A, eps, x_init=x_init.copy(), n_iters = n_iters, verbose=True)
    #s_iters, s_ranks = simple_cautiousrank(n, comps, eps, x_init=x_init.copy(), alpha=4, n_iters = n_iters, verbose=True)
    
    ham_results.append([hamdist(ranks[i,:], true_rank) for i in range(ranks.shape[0])])
    #skr_results.append([hamdist(s_ranks[i,:], true_rank) for i in range(ranks.shape[0])])
    
    
    k1dist_results.append([k_dist(ranks[i,:], true_rank, 1) for i in range(ranks.shape[0])])
    k5dist_results.append([k_dist(ranks[i,:], true_rank, 5) for i in range(ranks.shape[0])])
    k10dist_results.append([k_dist(ranks[i,:], true_rank, 10) for i in range(ranks.shape[0])])

In [ ]:
## hamming distance plot
plt.clf()
plot_average_w_ci(ham_results, x = range(n_iters+1), n_lines = 2, line_i = 0, label="KaczRank")
plt.xlabel("Iteration")
plt.ylabel("Hamming distance from true ranking")
plt.xlim(left=0, right=n_iters+1)
plt.tight_layout()
plt.legend()
plt.savefig("full_info_ham_dist_new.pdf")
plt.show()

In [ ]:
## k-distance plot
plt.clf()
plot_average_w_ci(k1dist_results, label = '$k=1$', line_i = 0, x = range(n_iters+1), n_lines = 3)
plot_average_w_ci(k5dist_results, label = '$k=5$', line_i = 1, x = range(n_iters+1), n_lines = 3)
plot_average_w_ci(k10dist_results, label = '$k=10$', line_i =2, x = range(n_iters+1), n_lines = 3)
plt.xlabel("Iteration")
plt.ylabel("k-distance from true ranking")
plt.legend()
plt.xlim(left=0, right=n_iters+1)
plt.tight_layout()
plt.savefig("full_info_k_dist.pdf")
plt.show()

In [ ]:
## Experiment 2 : KaczRank applied to incomplete information. A fraction p of the full info is sampled and KaczRank is applied for a fixed number of iterations.

n = 50
n_iters = 10000
n_trials = 20
eps = 1e-5
true_vals = np.random.rand(n)
true_rank = get_ranking(true_vals)
A = build_adj_mat(true_rank)
x_init=np.random.rand(n)

ham_results = []
k1dist_results = []
k5dist_results = []
k10dist_results = []
ps = np.linspace(0.05, 1, 50)

for i in tqdm(range(n_trials)):
    ham_temp = []
    k1_temp = []
    k5_temp = []
    k10_temp = []
    for p in ps:
        rows = np.random.choice(A.shape[0], size = np.floor(p*A.shape[0]).astype(int), replace=False)
        A_partial = A[rows,:]
        iters, ranks = kaczrank(A_partial, eps, x_init=x_init, n_iters = n_iters)
        ham_temp.append(hamdist(ranks[-1,:], true_rank))
        k1_temp.append(k_dist(ranks[-1,:], true_rank, 1))
        k5_temp.append(k_dist(ranks[-1,:], true_rank, 5))
        k10_temp.append(k_dist(ranks[-1,:], true_rank, 10))
    ham_results.append(ham_temp)
    k1dist_results.append(k1_temp)
    k5dist_results.append(k5_temp)
    k10dist_results.append(k10_temp)

In [ ]:
## hamming distance plot
plt.clf()
plot_average_w_ci(ham_results, x = ps, n_lines = 1)
plt.xlabel("Fraction of comparisons used")
plt.ylabel("Hamming distance from true ranking after 10000 iterations")
plt.xlim(left=0, right=1)
plt.tight_layout()
plt.savefig("partial_info_ham_dist.pdf")
plt.show()

In [ ]:
## k-distance plot
plt.clf()
plot_average_w_ci(k1dist_results, label = '$k=1$', line_i = 0, x = ps, n_lines = 3)
plot_average_w_ci(k5dist_results, label = '$k=5$', line_i = 1, x = ps, n_lines = 3)
plot_average_w_ci(k10dist_results, label = '$k=10$', line_i =2, x = ps, n_lines = 3)
plt.xlabel("Fraction of comparisons used")
plt.ylabel("$k$-distance from true ranking after 10000 iterations")
plt.legend()
plt.xlim(left=0, right=1)
plt.tight_layout()
plt.savefig("partial_info_k_dist.pdf")
plt.show()

In [ ]:
## Experiment 3 : All methods applied to noisy information. Each sample has probability p of being noisy.

n = 20
n_iters = 10000
n_trials = 20
eps = 1e-5
true_vals = np.random.rand(n)
true_rank = get_ranking(true_vals)
A = build_adj_mat(true_rank)
x_init=np.random.rand(n)

kr_ham_results = []
kr_k1dist_results = []
kr_k5dist_results = []
kr_k10dist_results = []

cr_ham_results = []
cr_k1dist_results = []
cr_k5dist_results = []
cr_k10dist_results = []

ps = np.linspace(0, 0.3, 50)

for i in tqdm(range(n_trials)):
    kr_ham_temp = []
    kr_k1_temp = []
    kr_k5_temp = []
    kr_k10_temp = []
    cr_ham_temp = []
    cr_k1_temp = []
    cr_k5_temp = []
    cr_k10_temp = []

    for p in tqdm(ps, leave=False):
        kr_iters, kr_ranks = noisykaczrank(A,p, eps, x_init=x_init, n_iters = n_iters)
        cr_iters, cr_ranks = cautiousrank(A,p,eps, x_init=x_init, alpha = 4, n_iters = n_iters)
        
        kr_ham_temp.append(hamdist(kr_ranks[-1,:], true_rank))
        kr_k1_temp.append(k_dist(kr_ranks[-1,:], true_rank, 1))
        kr_k5_temp.append(k_dist(kr_ranks[-1,:], true_rank, 5))
        kr_k10_temp.append(k_dist(kr_ranks[-1,:], true_rank, 10))
        cr_ham_temp.append(hamdist(cr_ranks[-1,:], true_rank))
        cr_k1_temp.append(k_dist(cr_ranks[-1,:], true_rank, 1))
        cr_k5_temp.append(k_dist(cr_ranks[-1,:], true_rank, 5))
        cr_k10_temp.append(k_dist(cr_ranks[-1,:], true_rank, 10))
        
    kr_ham_results.append(kr_ham_temp)
    kr_k1dist_results.append(kr_k1_temp)
    kr_k5dist_results.append(kr_k5_temp)
    kr_k10dist_results.append(kr_k10_temp)
    cr_ham_results.append(cr_ham_temp)
    cr_k1dist_results.append(cr_k1_temp)
    cr_k5dist_results.append(cr_k5_temp)
    cr_k10dist_results.append(cr_k10_temp)
    

In [ ]:
np.save('kr_ham_results.npy', kr_ham_results)
np.save('kr_k1dist_results.npy', kr_k1dist_results)
np.save('kr_k5dist_results.npy', kr_k5dist_results)
np.save('kr_k10dist_results.npy', kr_k10dist_results)
np.save('cr_ham_results.npy', cr_ham_results)
np.save('cr_k1dist_results.npy', cr_k1dist_results)
np.save('cr_k5dist_results.npy', cr_k5dist_results)
np.save('cr_k10dist_results.npy', cr_k10dist_results)

In [ ]:
if False:
    kr_ham_results = np.load('kr_ham_results.npy')
    kr_k1dist_results = np.load('kr_k1dist_results.npy')
    kr_k5dist_results = np.load('kr_k5dist_results.npy')
    kr_k10dist_results = np.load('kr_k10dist_results.npy')
    cr_ham_results = np.load('cr_ham_results.npy')
    cr_k1dist_results = np.load('cr_k1dist_results.npy')
    cr_k5dist_results = np.load('cr_k5dist_results.npy')
    cr_k10dist_results = np.load('cr_k10dist_results.npy')


In [ ]:
# FIG 6A

plt.clf()
plot_average_w_ci(kr_ham_results, line_i = 0, x = ps, n_lines = 1)
plt.xlabel("Probability of flipped comparison")
plt.ylabel("Hamming distance from true ranking after 10000 iterations")
plt.xlim(left=0, right=0.3)
plt.tight_layout()
plt.savefig("noisy_kr_hamdist.pdf")
plt.show()

In [ ]:
# FIG 6B

plt.clf()
plot_average_w_ci(cr_ham_results, line_i = 0, x = ps, n_lines = 1)
plt.xlabel("Probability of flipped comparison")
plt.ylabel("Hamming distance from true ranking after 10000 iterations")
plt.xlim(left=0, right=0.3)
plt.tight_layout()
plt.savefig("noisy_cr_hamdist.pdf")
plt.show()

In [ ]:
# Fig 6 combined
plt.clf()
plot_average_w_ci(kr_ham_results, line_i = 0, x = ps, n_lines = 2, label="KaczRank")
plot_average_w_ci(cr_ham_results, line_i = 1, x = ps, n_lines = 2, label="CautiousRank")
plt.xlabel("Probability of flipped comparison")
plt.ylabel("Hamming distance from true ranking")
plt.xlim(left=0, right=0.3)
plt.ylim(0,20.0)
plt.legend()
plt.tight_layout()
plt.savefig("noisy_kr_cr_hamdist.pdf")
plt.show()

In [ ]:
# FIG 7A

plt.clf()
plot_average_w_ci(kr_k1dist_results, label = '$k=1$', line_i = 0, x = ps, n_lines = 3)
plot_average_w_ci(kr_k5dist_results, label = '$k=5$', line_i = 1, x = ps, n_lines = 3)
plot_average_w_ci(kr_k10dist_results, label = '$k=10$', line_i =2, x = ps, n_lines = 3)
plt.xlabel("Probability of flipped comparison")
plt.ylabel("$k$-distance from true ranking")
plt.legend()
plt.xlim(left=0, right=0.3)
plt.ylim(0,20.0)
plt.tight_layout()
plt.savefig("noisy_kr_k_dist.pdf")
plt.show()

In [ ]:
# Fig 7B

plt.clf()
plot_average_w_ci(cr_k1dist_results, label = '$k=1$', line_i = 0, x = ps, n_lines = 3)
plot_average_w_ci(cr_k5dist_results, label = '$k=5$', line_i = 1, x = ps, n_lines = 3)
plot_average_w_ci(cr_k10dist_results, label = '$k=10$', line_i =2, x = ps, n_lines = 3)
plt.xlabel("Probability of flipped comparison")
plt.ylabel("$k$-distance from true ranking")
plt.legend()
plt.xlim(left=0, right=0.3)
plt.ylim(0,20.0)
plt.tight_layout()
plt.savefig("noisy_cr_k_dist.pdf")
plt.show()

In [ ]:
# Fig 7 combined

plt.clf()
plot_average_w_ci(kr_k1dist_results, label = 'KR $k=1$', line_i = 0, x = ps, n_lines = 6)
plot_average_w_ci(kr_k5dist_results, label = 'KR $k=5$', line_i = 1, x = ps, n_lines = 6)
plot_average_w_ci(kr_k10dist_results, label = 'KR $k=10$', line_i =2, x = ps, n_lines = 6)

plot_average_w_ci(cr_k1dist_results, label = 'CR $k=1$', line_i = 3, x = ps, n_lines = 6)
plot_average_w_ci(cr_k5dist_results, label = 'CR $k=5$', line_i = 4, x = ps, n_lines = 6)
plot_average_w_ci(cr_k10dist_results, label = 'CR $k=10$', line_i =5, x = ps, n_lines = 6)
plt.xlabel("Probability of flipped comparison")
plt.ylabel("$k$-distance from true ranking")
plt.legend()
plt.xlim(left=0, right=0.3)
plt.ylim(0,20.0)
plt.tight_layout()
plt.savefig("noisy_kr_cr_k_dist.pdf")
plt.show()

In [ ]:
## Experiment 4 : CautiousRank and FrequentRank applied to noisy, incomplete information.

n = 20
n_iters = 10000
n_trials = 20
eps = 1e-5
true_vals = np.random.rand(n)
true_rank = get_ranking(true_vals)
A = build_adj_mat(true_rank)
x_init=np.random.rand(n)

cr_ham_results = []
cr_k1dist_results = []
cr_k5dist_results = []
cr_k10dist_results = []

ps = np.linspace(0.05, 1, 50)
noisy_prob = 0.05

for i in tqdm(range(n_trials)):
    cr_ham_temp = []
    cr_k1_temp = []
    cr_k5_temp = []
    cr_k10_temp = []
    for p in ps:
        rows = np.random.choice(A.shape[0], size = np.floor(p*A.shape[0]).astype(int), replace=False)
        A_partial = A[rows,:]
        iters, ranks = cautiousrank(A_partial,noisy_prob, eps, x_init=x_init, alpha = 4, n_iters = n_iters)
        cr_ham_temp.append(hamdist(ranks[-1,:], true_rank))
        cr_k1_temp.append(k_dist(ranks[-1,:], true_rank, 1))
        cr_k5_temp.append(k_dist(ranks[-1,:], true_rank, 5))
        cr_k10_temp.append(k_dist(ranks[-1,:], true_rank, 10))
    cr_ham_results.append(cr_ham_temp)
    cr_k1dist_results.append(cr_k1_temp)
    cr_k5dist_results.append(cr_k5_temp)
    cr_k10dist_results.append(cr_k10_temp)

In [ ]:
plt.clf()
plot_average_w_ci(cr_ham_results, line_i = 0, x = ps, n_lines = 1)
plt.xlabel("Fraction of comparisons used")
plt.ylabel("Hamming distance from true ranking after 10000 iterations")
plt.xlim(left=0, right=1)
plt.tight_layout()
plt.savefig("noisy_incomplete_cr_005_hamdist.pdf")
plt.show()

In [ ]:
plt.clf()
plot_average_w_ci(cr_k1dist_results, label = '$k=1$', line_i = 0, x = ps, n_lines = 3)
plot_average_w_ci(cr_k5dist_results, label = '$k=5$', line_i = 1, x = ps, n_lines = 3)
plot_average_w_ci(cr_k10dist_results, label = '$k=10$', line_i =2, x = ps, n_lines = 3)
plt.xlabel("Fraction of comparisons used")
plt.ylabel("$k$-distance from true ranking after 10000 iterations")
plt.legend()
plt.xlim(left=0, right=1)
plt.tight_layout()
plt.savefig("noisy_incomplete_cr_005_kdist.pdf")
plt.show()

In [ ]:
## Experiment 6: Computing average number of iterations for convergence of KaczRank, for varying numbers of objects.

n_trials = 50
ns = [5,10,20,50,100,200]
eps = 1e-5

results = []

for n in tqdm(ns):
    temp_res = []
    eps = 1e-5
    true_vals = np.random.rand(n)
    true_rank = get_ranking(true_vals)
    A = build_adj_mat(true_rank)
    for trial in tqdm(range(n_trials)):
        ct = kr_itercount(A,eps,x_init=np.random.rand(n))
        temp_res.append(ct)
    results.append(temp_res)

In [ ]:
avg = sum(temp_res)/len(temp_res)

new_temp = temp_res
while len(new_temp) < 50:
    new_temp.append(avg)

In [ ]:
results_arr = np.array(results)

In [ ]:
sorted_res = [results_arr[:,i] for i in range(results_arr.shape[1])]

In [ ]:
plt.clf()
plot_average_w_ci(sorted_res, x = ns, n_lines = 1)
plt.xlabel('Number of objects $n$')
plt.ylabel('Number of iterations to converge')
plt.xlim(left=5, right=200)
plt.yscale('log')
plt.xscale('log')
plt.tight_layout()
plt.savefig("new_log_n_iters_vs_log_n_objects.pdf")

In [ ]:
## Experiment 7: Parameter Tuning for CautiousRank

n_iters = 10000
n = 40
eps = 1e-5
n_trials = 25
alphas = range(1,11)

results_005 = []
results_010 = []
results_020 = []

true_vals = np.random.rand(n)
true_rank = get_ranking(true_vals)
A = build_adj_mat(true_rank)
for trial in tqdm(range(n_trials)):
    x_init=np.random.rand(n)
    trial_res = []
    for alpha in alphas:
        iters, ranks = cautiousrank(A,0.05, eps, x_init=x_init, alpha =alpha, n_iters = n_iters)
        trial_res.append(hamdist(ranks[-1,:], true_rank))
    results_005.append(trial_res)

for trial in tqdm(range(n_trials)):
    x_init=np.random.rand(n)
    trial_res = []
    for alpha in alphas:
        iters, ranks = cautiousrank(A,0.1, eps, x_init=x_init, alpha =alpha, n_iters = n_iters)
        trial_res.append(hamdist(ranks[-1,:], true_rank))
    results_010.append(trial_res)

for trial in tqdm(range(n_trials)):
    x_init=np.random.rand(n)
    trial_res = []
    for alpha in alphas:
        iters, ranks = cautiousrank(A,0.2, eps, x_init=x_init, alpha =alpha, n_iters = n_iters)
        trial_res.append(hamdist(ranks[-1,:], true_rank))
    results_020.append(trial_res)

In [ ]:
results_005_arr = np.array(results_005)
results_010_arr = np.array(results_010)
results_020_arr = np.array(results_020)

listed_results_005 = [results_005_arr[i,:] for i in range(results_005_arr.shape[0])]
listed_results_010 = [results_010_arr[i,:] for i in range(results_010_arr.shape[0])]
listed_results_020 = [results_020_arr[i,:] for i in range(results_020_arr.shape[0])]

In [ ]:
plt.clf()
plot_average_w_ci(listed_results_005, x = alphas, label = '$p = 0.05$', line_i = 0, n_lines = 3)
plot_average_w_ci(listed_results_010, x = alphas, label = '$p = 0.1$', line_i = 1, n_lines = 3)
plot_average_w_ci(listed_results_020, x = alphas, label = '$p = 0.2$', line_i = 2, n_lines = 3)
plt.xlabel('Cautiousness parameter')
plt.ylabel('Hamming distance from true ranking after 10000 iterations')
plt.xlim(left=1, right=10)
plt.legend()
plt.tight_layout()
plt.savefig("hamdist_vs_alpha_cr_n_40.pdf")

In [ ]:
## Last experiment: A comparison of all distance metrics

n = 50
n_iters = 10000
n_trials = 20
eps = 1e-5
true_vals = np.random.rand(n)
true_rank = get_ranking(true_vals)
A = build_adj_mat(true_rank)
x_init=np.random.rand(n)

ham_results = []
k5dist_results = []
k10dist_results = []
ken_tau_results = []
cayley_results = []

def normalised_cayley_distance(x,y):
    B = range(len(x))
    inv_y = tuple(y.index(a) for a in B)
    comp = tuple(x[inv_y[a]] for a in B)
    cycles = 0
    rem = set(B)
    while rem:
        a = rem.pop()
        cycles += 1
        while comp[a] in rem:
            a = comp[a]
            rem.remove(a)
    return (len(B) - cycles)/(n-1)

def normalised_kendall_tau_distance(values1, values2):
    """Compute the Kendall tau distance."""
    n = len(values1)
    assert len(values2) == n, "Both lists have to be of equal length"
    i, j = np.meshgrid(np.arange(n), np.arange(n))
    a = np.argsort(values1)
    b = np.argsort(values2)
    ndisordered = np.logical_or(np.logical_and(a[i] < a[j], b[i] > b[j]), np.logical_and(a[i] > a[j], b[i] < b[j])).sum()
    return ndisordered / (n * (n - 1))

for i in tqdm(range(n_trials)):
    iters, ranks = kaczrank(A, eps, x_init=x_init, n_iters = n_iters)
    ham_results.append([hamdist(ranks[i,:], true_rank)/n for i in range(ranks.shape[0])])
    k5dist_results.append([k_dist(ranks[i,:], true_rank, 5)/n for i in range(ranks.shape[0])])
    k10dist_results.append([k_dist(ranks[i,:], true_rank, 10)/n for i in range(ranks.shape[0])])
    ken_tau_results.append([normalised_kendall_tau_distance(list(true_vals), list(iters[i,:])) for i in range(ranks.shape[0])])
    cayley_results.append([normalised_cayley_distance(list(ranks[i,:].astype(int)), list(true_rank.astype(int))) for i in range(ranks.shape[0])])


In [ ]:
plt.clf()
plot_average_w_ci(ham_results, x = range(n_iters+1), label = 'Hamming', line_i = 0, n_lines = 5)
plot_average_w_ci(k5dist_results, x = range(n_iters+1), label = '$5$-distance', line_i = 1, n_lines = 5)
plot_average_w_ci(k10dist_results, x =range(n_iters+1), label = '$10$-distance', line_i = 2, n_lines = 5)
plot_average_w_ci(ken_tau_results, x = range(n_iters+1), label = 'Kendall Tau', line_i = 3, n_lines = 5)
plot_average_w_ci(cayley_results, x = range(n_iters+1), label = 'Cayley', line_i = 4, n_lines = 5)
plt.xlabel('Iteration')
plt.ylabel('Distance from true ranking')
plt.xlim(left=1, right=n_iters)
plt.legend()
plt.tight_layout()
plt.savefig("all_distances.pdf")

In [ ]:
## Experiment 8: Cautiousrank time vs. other methods
import time
import tracemalloc

n_iters = 5*10**6
ns = np.logspace(np.log10(50), 3.0,25)

eps = 1e-5
n_trials = 25

In [ ]:
n_methods = 4
methods = ["KaczRank", "LSR Pairwise", "I-LSR Pairwise", "Rank Centrality"]

dists = np.zeros((len(ns), n_trials, n_methods))
times = np.zeros((len(ns), n_trials, n_methods))
memory = np.zeros((len(ns), n_trials, n_methods))
iters_per = np.zeros((len(ns), n_trials))

# Small fixed delay to make timing more consistent
buffer = 1.0


for i,n in tqdm(enumerate(ns), total=len(ns)):
    n = int(n)
    true_vals = np.linspace(0,1,n)
    true_rank = get_ranking(true_vals)

    A,comps = build_adj_mat(true_rank, make_comps=True, p=0.00)
    

    
    for j, trial in tqdm(enumerate(range(n_trials)), leave=False, total=n_trials):

        x_init=np.random.rand(n)
        
        # KaczRank
        tracemalloc.start()
        start = time.time()
        
        kr_rank, iters_out = kaczrank(A,eps,x_init=x_init.copy(),n_iters = 10**8, verbose=False, check_convergence=True)
        time.sleep(buffer)
        memory[i,j,0] = tracemalloc.get_traced_memory()[1]
        
        end = time.time()
        tracemalloc.stop()
        
        
        kr_rank = get_ranking(kr_rank)
        dists[i,j,0] = hamdist(kr_rank, true_rank)
        times[i,j,0] = (end - start - buffer)
        
        iters_per[i,j] = iters_out
        
        
        # LSR Pairwise
        tracemalloc.start()
        start = time.time()
        
        rc_rank = lsr_pairwise(n, comps, alpha=1e-4)#, max_iter = 1000)
        memory[i,j,1] = tracemalloc.get_traced_memory()[1]
        time.sleep(buffer)
        end = time.time()
        tracemalloc.stop()
        
        rc_rank = get_ranking(-rc_rank)
        dists[i,j,1] = k_dist(rc_rank, true_rank,5)
        times[i,j,1] = end - start - buffer
        
        # ILSR Pairwise
        tracemalloc.start()
        start = time.time()
        
        rc_rank = ilsr_pairwise(n, comps, alpha=1e-4, max_iter = 1000)
        memory[i,j,2] = tracemalloc.get_traced_memory()[1]
        time.sleep(buffer)
        end = time.time()
        tracemalloc.stop()
        
        rc_rank = get_ranking(-rc_rank)
        dists[i,j,2] = k_dist(rc_rank, true_rank,5)
        times[i,j,2] = end - start - buffer
        
        # Rank Centrality
        tracemalloc.start()
        start = time.time()
        
        rc_rank = rank_centrality(n, comps, alpha=1e-4)#, max_iter = 1000)
        memory[i,j,3] = tracemalloc.get_traced_memory()[1]
        time.sleep(buffer)
        end = time.time()
        tracemalloc.stop()
        
        rc_rank = get_ranking(-rc_rank)
        dists[i,j,3] = k_dist(rc_rank, true_rank,5)
        times[i,j,3] = end - start - buffer
        
        np.save(f"last_experiment/experiment_{int(time.time())}.npy",[dists, times, memory])

In [ ]:
%matplotlib inline
plt.clf()
fig, ax = plt.subplots(1,2, dpi=600, figsize=(10,5))
for i in range(n_methods):
    plot_average_w_ci(times[:,:,i].T, x = ns, label = methods[i], line_i = i, n_lines = n_methods, ax=ax[0])
#plot_average_w_ci(kr_times.T, x = ns, label = 'KaczRank', line_i = 0, n_lines = 2, ax=ax[0])
#plot_average_w_ci(rc_times.T, x = ns, label = 'LSR Pairwise', line_i = 1, n_lines = 2, ax=ax[0])
ax[0].set_xlabel("$n$")
ax[0].set_ylabel("Computational time to convergence")
ax[0].semilogx()
ax[0].semilogy()

for i in range(n_methods):
    plot_average_w_ci(memory[:,:,i].T, x = ns, label = methods[i], line_i = i, n_lines = n_methods, ax=ax[1])
#plot_average_w_ci(kr_mem.T, x = ns, label = 'KaczRank', line_i = 0, n_lines = 2, ax=ax[1])
#plot_average_w_ci(rc_mem.T, x = ns, label = 'LSR Pairwise', line_i = 1, n_lines = 2, ax=ax[1])
ax[1].set_xlabel("$n$")
ax[1].set_ylabel("Max. memory usage")
ax[1].semilogx()
ax[1].semilogy()
ax[1].legend()


plt.tight_layout()
plt.savefig("kr_vs_all_mem.pdf")


In [ ]:
%matplotlib inline
plt.clf()
for i in range(n_methods):
    plot_average_w_ci(times[:,:,i].T, x = ns, label = methods[i], line_i = i, n_lines = n_methods)
#plot_average_w_ci(kr_times.T, x = ns, label = 'KaczRank', line_i = 0, n_lines = 2, ax=ax[0])
#plot_average_w_ci(rc_times.T, x = ns, label = 'LSR Pairwise', line_i = 1, n_lines = 2, ax=ax[0])
plt.xlabel("$n$")
plt.ylabel("Computational time to convergence")
plt.semilogx()
plt.semilogy()
plt.legend()
plt.tight_layout()
plt.show()
#plt.savefig("new_kr_vs_all_time.pdf")


In [ ]:
%matplotlib inline
plt.clf()
for i in range(n_methods):
    plot_average_w_ci(memory[:,:,i].T, x = ns, label = methods[i], line_i = i, n_lines = n_methods)
#plot_average_w_ci(kr_times.T, x = ns, label = 'KaczRank', line_i = 0, n_lines = 2, ax=ax[0])
#plot_average_w_ci(rc_times.T, x = ns, label = 'LSR Pairwise', line_i = 1, n_lines = 2, ax=ax[0])
plt.xlabel("$n$")
plt.ylabel("Max. memory usage")
plt.semilogx()
plt.semilogy()
plt.legend()
plt.tight_layout()
plt.savefig("new_kr_vs_all_memory.pdf")